# Below code for performing actions on training data

In [ ]:
import os
import pandas as pd
import numpy as np
import random
import pickle


data_dir="C:/Users/DELL/Documents/SAMPLE_DATA_LETTERS_PLX_FOR_ML/newletters29_10_19/"
fold_in_dir=os.listdir(data_dir)
count=0
complete_file=[]
empty_row=pd.Series([])
total_rows_in_each=111
#class_names=['a','b','c','d','e']
training_data=[]
rowsshape=[]
class_names=[]

for i in fold_in_dir:
    files_in_fold=os.listdir(os.path.join( data_dir + i ))
    for j in files_in_fold:
        #no use of complete file ....used only read_file
        complete_file.append(os.path.join(data_dir + i+'/' + j))
        rowsshape.append((pd.read_csv((os.path.join(data_dir + i+'/' + j))).shape[0]))
        class_names.append(j[9:10])
maxrows=max(rowsshape)
minrows=min(rowsshape)
print(maxrows)
print(minrows)
                         



for i in range(len(complete_file)):
     
    read_file=pd.read_csv(f"{complete_file[i]}")

    read_file=read_file.drop(['Mx','My','Mz','T'],axis=1)
    norm_data=read_file

    
    for z in range(maxrows-(read_file.shape[0])):
        norm_data=norm_data.append(empty_row,ignore_index=True)
    
    no_nan_val=np.nan_to_num(norm_data)
   
    training_data.append([no_nan_val,class_names[i]])
    



random.shuffle(training_data)
features = []
labels = []
for r,s in training_data:
    features.append(r)
    labels.append(s) 

xmax = []
for xx in range(len(features)):
    xm = np.reshape(features[xx],(norm_data.shape[0]*norm_data.shape[1]))
    xmax.append(xm)
    
pickle_out = open('xnew.pickle','wb')
pickle.dump(xmax,pickle_out)
pickle_out.close()


pickle_out = open('ynew.pickle','wb')
pickle.dump(labels,pickle_out)
pickle_out.close()



# Training the model with training data

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import scipy.stats as s
import pandas as pd
from sklearn.svm import SVC
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix 

import pickle
featu = pickle.load(open('xnew.pickle','rb'))
label = pickle.load(open('ynew.pickle','rb'))

X_train, X_test, y_train, y_test = train_test_split(featu, label, random_state = 0) 
  
from sklearn.svm import SVC 
svm_model_linear = SVC().fit(X_train, y_train)
svm_predictions = svm_model_linear.predict(X_test) 
accuracy = svm_model_linear.score(X_test, y_test) 
print(accuracy)

rbf_svc = svm.SVC(kernel='poly')
rbfmodel=rbf_svc.fit(X_train, y_train)
rbfpred=rbfmodel.predict(X_test)
acc_rbf=rbfmodel.score(X_test,y_test)
print(acc_rbf)




cm = confusion_matrix(y_test, svm_predictions) 


# Code to test the model 

In [ ]:
import serial
import time
import csv
import os
ser=serial.Serial("COM6",38400)
empty_row=pd.Series([])



save_path ='C:/Users/DELL/Documents/SAMPLE_DATA_LETTERS_PLX_FOR_ML/'
#completeName = os.path.join(save_path, name_of_file+".txt")  

for i in range(1,20):
    
    #newfilename="C:\Users\DELL\Documents\SAMPLE_DATA_LETTERS_PLX_FOR_ML\newletters29_10_19\lett_a\data_let_a{}.csv".format(i)
    newfilename=os.path.join(save_path,'test_sample{}.csv'.format(i))
    #print(newfilename)
    x=(str(ser.readline()))[2:-5]
    #print(x)
    if(x=="START"):
        with open(newfilename,"w") as myfile:

            wr=csv.writer(myfile,delimiter=",")
            wr.writerow(["Ax","Ay","Az","Gx","Gy","Gz","Mx","My","Mz","T"])
            cc=(str(ser.readline()))[2:-5]
            while (cc!="END"):
                try:
                    #cc=str(ser.readline())
                    #p=(cc[2:-5]).split(" ")
                    p=cc.split(" ")
                    wr.writerow(p)
                    #print (p)
                    cc=(str(ser.readline()))[2:-5]
                except:
                    print(" keyboard interrupt")
                    myfile.close()
                    break
                    
        test_df=pd.read_csv(r"C:/Users/DELL/Documents/SAMPLE_DATA_LETTERS_PLX_FOR_ML/test_sample{}.csv".format(i))
        test_df=test_df.drop(['Mx','My','Mz','T'],axis=1)
        norm_t_data=test_df.iloc[0:233,:]
        #print(test_df.shape)
        #print(norm_t_data.shape)
        for z in range(maxrows-(test_df.shape[0])):
            norm_t_data=norm_t_data.append(empty_row,ignore_index=True)
        #print(norm_t_data.shape)
        non_nan_data=np.nan_to_num(norm_t_data)
        final_test=np.reshape(non_nan_data,(1,-1))
        new_predic=svm_model_linear.predict(final_test) 
        
        print(new_predic[0],end='')
        